In [1]:
import pandas as pd
import time
import plotly.graph_objs as go

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

print(__version__)

from ipywidgets import widgets
from plotly.widgets import GraphWidget
from IPython.display import display, clear_output, Image

init_notebook_mode(connected=True)

2.7.0


<IPython.core.display.Javascript object>

In [2]:
df = pd.read_csv('diversity_data.csv', sep=';', header=0)

In [17]:
df.dtypes

row_id                int64
project_id            int64
owner_login          object
name                 object
language             object
created_at           object
domain               object
forks                 int64
watchers              int64
total_committers      int64
total_commits         int64
project_age           int64
windows               int64
window_idx            int64
num_commits           int64
num_pull_req          int64
num_comments          int64
num_issues            int64
num_committers        int64
num_team              int64
committers           object
commits              object
team                 object
project_tenures      object
github_tenures       object
commit_tenures       object
genders              object
countries            object
left                 object
stayed               object
joined               object
male                  int64
female                int64
unknown               int64
has_woman              bool
f_known_gender      

In [23]:
filteredDF = df.iloc[:,[3,4,9,13,37,38]]
filteredDF.dtypes

name                 object
language             object
total_committers      int64
window_idx            int64
blau_gender         float64
blau_country        float64
dtype: object

In [27]:
filteredDF.head()

,name,language,total_committers,window_idx,blau_gender,blau_country
0,attribute_checkbox,PHP,6,1,0.000,0.0
1,attribute_checkbox,PHP,6,2,0.375,0.0
2,attribute_checkbox,PHP,6,4,0.000,0.0
3,attribute_checkbox,PHP,6,5,0.000,0.0
4,soundcloud-mac-sharing,Objective-C,4,1,0.000,0.0


In [28]:
grouped = filteredDF.groupby(['language','window_idx']).mean()
grouped.head()

total_committers  blau_gender  blau_country
language window_idx                                             
ASP      1                  12.625000     0.000000      0.218875
         2                  14.000000     0.000000      0.216429
         3                  23.000000     0.026667      0.203667
         4                  32.500000     0.000000      0.305500
         5                  22.666667     0.000000      0.370333

In [42]:
popular = list(filteredDF.groupby(['language']).sum().sort_values(by='total_committers',ascending=False).index.unique())
popular

['Ruby',
 'Python',
 'JavaScript',
 'C',
 'C++',
 'Java',
 'PHP',
 'Shell',
 'None',
 'Perl',
 'Emacs Lisp',
 'Erlang',
 'CoffeeScript',
 'Objective-C',
 'C#',
 'Clojure',
 'Puppet',
 'Haskell',
 'Go',
 'VimL',
 'Scala',
 'Groovy',
 'Lua',
 'OCaml',
 'CSS',
 'Vala',
 'TypeScript',
 'Common Lisp',
 'XML',
 'R',
 'Julia',
 'Rust',
 'FORTRAN',
 'Visual Basic',
 'D',
 'Coq',
 'Parrot',
 'ActionScript',
 'PowerShell',
 'BlitzBasic',
 'Augeas',
 'ColdFusion',
 'ASP',
 'Smalltalk',
 'Matlab',
 'Dart',
 'Scheme',
 'Dylan',
 'Racket',
 'Elixir',
 'HaXe',
 'Scilab',
 'Assembly',
 'Standard ML',
 'Verilog',
 'XC',
 'Prolog',
 'F#',
 'XSLT',
 'Arduino',
 'Ceylon',
 'XQuery',
 'Delphi',
 'Tcl',
 'Kotlin',
 'Haxe',
 'VHDL',
 'Objective-J',
 'TeX',
 'Boo',
 'Eiffel',
 'Processing',
 'Apex',
 'AppleScript',
 'Logos',
 'Crystal',
 'LiveScript',
 'Ecl',
 'OpenEdge ABL',
 'Volt',
 'Io',
 'SuperCollider',
 'Ada',
 'ooc',
 'Opa',
 'Pure Data',
 'Nimrod',
 'XProc',
 'Bro',
 'DOT',
 'Xtend',
 'Monkey',
 'Sla

In [43]:
language = widgets.Dropdown(
    options=popular,
    value='PHP',
    description='Language:'
)

g = GraphWidget('https://plot.ly/~cesarsl/3/')

In [46]:
def validate():
    if language.value in filteredDF['language'].unique():
        return True
    else:
        return False
    
def response(change):
    if validate():
        temp_df = grouped.loc[language.value]
        
        s = {
                'data': {
                    'x': [temp_df.index, temp_df.index],
                    'y': [temp_df.blau_gender, temp_df.blau_country],
                    'name': ['Blau Gender', 'Blau Country'],
                    'marker.color': ['rgb(178,18,18)','rgb(9,113,178)'],
                    'marker.size': 10,
                    'marker.symbol': 'hexagon-open',
                    'line.color': ['rgb(178,18,18)','rgb(9,113,178)'],
                    'line.width': 3,
                    'mode': 'line'
                },
                'traces': [0,1],
                'layout': {
                    'title': 'Gender and Nationality Diversity',
                    'showlegend':  True,
                    'xaxis': {'range': [1,24]},
                    'yaxis': {'range': [0,1]}
                }
                
            }
        g.restyle(s['data'], indices=s['traces'])
        g.relayout(s['layout'])
            
language.observe(response, names='value')

In [47]:
container2 = widgets.HBox([language])
display(container2)
display(g)

GraphWidget()